In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
rng = np.random.default_rng(seed = 0)

In [3]:
accent_list = [
    "african",  "australia", "bermuda",
    "canada", "england", "hongkong",
    "indian", "ireland", "malaysia", 
    "newzealand", "philippines", "scotland",
    "singapore", "southatlandtic", "us", 
    "wales",
]

In [4]:
## Uncomment this cell only when you explicitly require it.
## This creates files/dirs so be careful
## Chandra
# for accent in accent_list:
#    os.system(f"mkdir -p ../data/{accent}/")
#    os.system(f"cp ../../../mz-isca/validated/accent-trans/{accent}.json ../data/{accent}/all_validated.json")

In [5]:
def _write(path, json_list):
    with open(path, 'w') as file:
        for json_entry in json_list:
            json.dump(json_entry, file)
            file.write("\n")

In [19]:
## Exec this cell with care
## As this creates/modifies some files
## Chandra


for accent in accent_list:
    all_validated_json_path = f"../data/{accent}/all_validated.json"

    with open(all_validated_json_path, 'r') as all_validated_json_file:
        json_lst = [json.loads(line.strip()) for line in all_validated_json_file.readlines()]

        rng.shuffle(json_lst)

        all_json_sz = min(8000, len(json_lst))
        all_json_inds = rng.choice(len(json_lst), all_json_sz, replace=False)
        all_json_lst = [json_lst[ind] for ind in all_json_inds]

        all_json_path = f"../data/{accent}/all.json"
        _write(all_json_path, all_json_lst)
    #     json_lst = [json.loads(line.strip()) for line in all_json_file.readlines()]

    #     # print("Old json lst size", len(json_lst))

    #     rng.shuffle(json_lst)


    #     dev_path = f"../data/{accent}/dev.json"
    #     dev_sz = 50

    #     dev_lst = rng.choice(len(json_lst), dev_sz, replace=False)

    #     # print("dev _lst", len(dev_lst))
    #     dev_json_items = [json_lst[_] for _ in dev_lst]

    #     _write(dev_path, dev_json_items)

    #     json_lst = [json_lst[ind] for ind in range(len(json_lst)) if ind not in dev_lst]

    #     # print("new json lst size", len(json_lst))



    #     for json_entry in json_lst:
    #         speaker_id = json_entry["client_id"]
    #         if speaker_id in speaker_dict.keys():
    #             speaker_dict[speaker_id].append(json_entry)
    #         else:
    #             speaker_dict[speaker_id] = [json_entry]
    
    # print(accent, len(speaker_dict.keys()))


    # sorted_speakers = list(speaker_dict.keys())

    # sorted_speakers.sort(key=lambda user_id: len(speaker_dict[user_id]), reverse = True)

    # total_speakers = len(sorted_speakers)
    # lower = total_speakers//4
    # upper = total_speakers * 3//4

    # reqd_user_cnt = max(2, (upper - lower + 1)*2//10) # 0.1% of users are selected

    # assert reqd_user_cnt <= upper-lower + 1

    # test_user_ids = rng.choice(sorted_speakers[lower: upper + 1], size=reqd_user_cnt, replace=False)


    # test_path = f"../data/{accent}/test.json"
    # selection_path = f"../data/{accent}/selection.json"

    # test_json_list = []
    # selection_json_list = []

    # for user_id in sorted_speakers:
        
    #     if user_id in test_user_ids:
    #         test_json_list.extend(speaker_dict[user_id])            
    #     else:
    #         selection_json_list.extend(speaker_dict[user_id])

    # rng.shuffle(test_json_list)
    # rng.shuffle(selection_json_list)

    # _write(test_path, test_json_list)

    # _write(selection_path, selection_json_list)


In [6]:
# TODO: need to run the mp3wav.py
# Run the below command in a seperate bash shell instead of here
# !pwd; cd ../; pwd; which python; python scripts/mp3wav.py;

In [7]:
for accent in accent_list:
    all_json_path = f"../data/{accent}/all.json"
    
    with open(all_json_path, 'r') as all_json_file:
        json_lst = [json.loads(line.strip()) for line in all_json_file.readlines()]
        rng.shuffle(json_lst)
        speaker_dict = dict()
        for item in json_lst:
            speaker_id = item["client_id"]
            if speaker_id not in speaker_dict.keys():
                speaker_dict[speaker_id] = [item]
            else:
                speaker_dict[speaker_id].append(item)

        print(accent, f"num_speakers = {len(speaker_dict.keys())}", f"all_json_count: {len(json_lst)}")

        sorted_speakers = list(speaker_dict.keys())

        sorted_speakers.sort(key = lambda speaker_id: len(speaker_dict[speaker_id]), reverse = True)

        total_speakers = len(sorted_speakers)
        lower = total_speakers//4
        upper = total_speakers * 3//4

        test_speaker_count = max(2, int(0.3 * total_speakers))
        
        test_speakers = rng.choice(sorted_speakers[lower: upper + 1], test_speaker_count, replace = False)

        test_json_lst = []
        for test_user in test_speakers:
            test_json_lst.extend(speaker_dict[test_user])
        
        rng.shuffle(test_json_lst)

        test_json_path = f"../data/{accent}/test.json"
        print(accent, f"test_speakers = {test_speaker_count}", f"test_json_count: {len(test_json_lst)}")

        _write(test_json_path, test_json_lst)


        rem_speakers = [speaker for speaker in sorted_speakers if speaker not in test_speakers]

        rng.shuffle(rem_speakers)
        print(accent, f"remaining speakers after removing test_entries = {len(rem_speakers)}")

        for speaker in rem_speakers:
            rng.shuffle(speaker_dict[speaker])
        
        seed_cnt = 50
        seed_lst = []

        for curr_ind in range(seed_cnt):
            speaker_id = rem_speakers[(curr_ind)%len(rem_speakers)]

            seed_lst.append(speaker_dict[speaker_id][-1])

            speaker_dict[speaker_id].pop()

            if len(speaker_dict[speaker_id]) == 0:
                assert speaker_id in rem_speakers
                rem_speakers.remove(speaker_id)

        print(accent, f"seed_json_count: {len(seed_lst)}")
        print(accent, f"remaining speakers after removing seed_entries = {len(rem_speakers)}")

        seed_path = f"../data/{accent}/seed.json"
        _write(seed_path, seed_lst)

        rng.shuffle(rem_speakers)
        left_samples = 0

        for speaker in rem_speakers:
            rng.shuffle(speaker_dict[speaker])
            left_samples += len(speaker_dict[speaker])
            assert len(speaker_dict[speaker]) != 0

        print(accent, f"remaining samples after removing seed_entries = {left_samples}")

        dev_cnt = int(0.1 * left_samples)

        dev_lst = []

        for curr_ind in range(dev_cnt):
            speaker_id = rem_speakers[(curr_ind)%len(rem_speakers)]

            dev_lst.append(speaker_dict[speaker_id][-1])

            speaker_dict[speaker_id].pop()

            if len(speaker_dict[speaker_id]) == 0:
                assert speaker_id in rem_speakers
                rem_speakers.remove(speaker_id)


        print(accent, f"dev_json_count: {len(dev_lst)}")
        print(accent, f"remaining speakers after removing dev_entries = {len(rem_speakers)}")

        dev_path = f"../data/{accent}/dev.json"
        _write(dev_path, dev_lst)

        selection_path = f"../data/{accent}/selection.json"
        selection_lst = []

        for speaker in rem_speakers:
            selection_lst.extend(speaker_dict[speaker])

        print(accent, f"selection_json_count: {len(selection_lst)}")

        _write(selection_path, selection_lst)

        






african num_speakers = 229 all_json_count: 8000
african test_speakers = 68 test_json_count: 1037
african remaining speakers after removing test_entries = 161
african seed_json_count: 50
african remaining speakers after removing seed_entries = 159
african remaining samples after removing seed_entries = 6913
african dev_json_count: 691
african remaining speakers after removing dev_entries = 105
african selection_json_count: 6222
australia num_speakers = 498 all_json_count: 8000
australia test_speakers = 149 test_json_count: 713
australia remaining speakers after removing test_entries = 349
australia seed_json_count: 50
australia remaining speakers after removing seed_entries = 339
australia remaining samples after removing seed_entries = 7237
australia dev_json_count: 723
australia remaining speakers after removing dev_entries = 189
australia selection_json_count: 6514
bermuda num_speakers = 41 all_json_count: 643
bermuda test_speakers = 12 test_json_count: 100
bermuda remaining speakers

In [22]:
# from pydub import AudioSegment
# for accent in accent_list:
#     for file_name in ["selection", "test", "dev", "all"]:
#         print(accent, file_name, "Started")
#         file_path = f"../data/{accent}/{file_name}.json"
#         with open(file_path, 'r') as file:
#             json_lst = [json.loads(line.strip()) for line in file.readlines()]

#             new_json_lst = []
#             for entry in json_lst:
#                 new_entry = entry
#                 try:
#                     audio = AudioSegment.from_file(entry["audio_filepath"])
#                     new_entry["duration"] = audio.duration_seconds
#                     new_json_lst.append(new_entry)
#                 except:
#                     print(entry["audio_filepath"])

#         _write(file_path, new_json_lst)



#         print(accent, file_name, "Done")